In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
#RegEx
import re

import scrubadub
#import scrubadub[address]




VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1608514887400_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  r.review_headline, r.review_comments, r.reviewer_nickname, r.reviewer_location, \
case when (contains(r.observations, 'PII') )  then 1 else 0 end as PII \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US') and contains(r.observations, 'PM') and mg.name not ilike '%hartford%' \
and r.created_date > '2020-01-01' "


#and contains(r.observations, 'PII') \
#and ugc_id in (270519748, 202761051, 202759202, 203453834, 135796841, 135854691, 217517666, 275335901, 264483850, 175108572, 175148446) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head(20)
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                  REVIEW_HEADLINE  \
0   280813281  Delicious Parker House Rolls                      
1   280711252  The artificial scent is too strong.               
2   280716079  Laundry Detergent!                                
3   280738851  Love them!                                        
4   280713287  Good product.                                     
5   280784686  Awesome                                           
6   280805536  Poor choice for a plant hanger                    
7   280727449  nice watch                                        
8   280753843  Lots of short stories about a half hour each      
9   280778270  Privacy screen - does what it's supposed to.      
10  280814897  Awesome                                           
11  280791539  So many uses!                                     
12  280716109  New Flower Beds                                   
13  280751941  Looks better as expected!                         
14  280742

In [17]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16550384 entries, 0 to 16550383
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   UGC_ID             int64 
 1   REVIEW_HEADLINE    object
 2   REVIEW_COMMENTS    object
 3   REVIEWER_NICKNAME  object
 4   REVIEWER_LOCATION  object
 5   PII                int64 
dtypes: int64(2), object(4)
memory usage: 757.6+ MB

In [18]:
#Force space at the end so that regex will match space after location
#This is necessary so if they don't use a zip code and the address ends in just the state abbreviation, it will pick it up 
#But will not pick up in text where something is a word that starts with a state's 2 letter abbreviation and thus creates lots of false positives

reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS'] + ' '+reviews['REVIEWER_NICKNAME'] + ' '+reviews['REVIEWER_LOCATION'] +' '
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                      REVIEW_HEADLINE  \
0  280813281  Delicious Parker House Rolls          
1  280711252  The artificial scent is too strong.   
2  280716079  Laundry Detergent!                    
3  280738851  Love them!                            
4  280713287  Good product.                         

                                                                                                                                                                                                                                                                                                                                                                         REVIEW_COMMENTS  \
0  Sister Schubert has the best Parker House Style Yeast Rolls. They come in a aluminum pan., cover it up with aluminum foil and bake them in the oven. They always taste like they are homemade. We eat these several times a week. They are definitely a  part of our family meals.                        

In [19]:
reviews['regex_cc'] = reviews['REVIEW_COMMENTS'].str.contains(r'\d{4}(\s|-|\.|\*)\d{4}(\s|-|\.|\*)\d{4}(\s|-|\.|\*)\d{4}')== True

reviews['regex_socsec']  = reviews['ALL_TEXT'].str.contains(r'\b\d{3}(\s|-|\.|\*)\d{2}(\s|-|\.|\*)\d{4}\b')== True

reviews['regex_phone10'] = reviews['ALL_TEXT'].str.contains(r'(\d(\s|-|\.|\*)?)?(\(?\d{3}\)?|\d{3})(\s|-|\.|\*)\d{3}(\s|-|\.|\*)\d{4}')== True

reviews['regex_email'] = reviews['ALL_TEXT'].str.contains(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}', case=False)== True


#Handle regex will pick up majority of email addresses
reviews['regex_handle'] = reviews['ALL_TEXT'].str.contains(r'@(?!influenster|voxbox|Buchanan\s?Group|PINCHme|SampleSource|ChickAdvisor|BrandShare|SoPost|Sampler)[A-Za-z0-9._]+', case=False)== True

#MUST MAKE CASE INSENSITIVE
#Apply to each text field separately bc otherwise if apply to ALL_TEXT then can match # from comments and word from nickname
#and city, state from location and get a match
reviews['regex_USaddress_loc'] = reviews['REVIEWER_LOCATION']. \
str.contains(r'\d{1,6}(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)\s(STREET|ST|DRIVE|DR|AVENUE|AVE|ROAD|RD|LOOP|COURT|CT|CIRCLE|LANE|LN|BOULEVARD|BLVD|COVE|TERRACE|TERR|TER|PLACE)\.?\,?(\s(APARTMENT|APT|BUILDING|BLDG|DEPARTMENT|DEPT|FLOOR|FL|LOT|PIER|ROOM|RM|TRAILER|TRLR|UNIT|PO BOX)\.?\s\#?[A-Za-z0-9]+\,?)?(\s[A-Za-z]+)?(\s[A-Za-z]+)\,?\s(A[LKSZR]|C[AOT]|D[EC]|F[L]|G[AU]|HI|I[ADLN]|K[SY]|LA|M[ADEINOPST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])(\s\d{5})?(\-\d{4})?', case=False)== True

#Option to require word boundary after state code in the comments field since such long text, not necessary in location or headline field
#No word boundary will match "50 mile drive one way...." bc word "way" contains WA as in Washington state
#HOWEVER then we are losing if it was legit address but they decided to type out Washington instead of just WA
#For now, no word boundary required since more likely to write out state than for it to just so happen to match another random word in text
reviews['regex_USaddress_comments'] = reviews['REVIEW_COMMENTS']. \
str.contains(r'\d{1,6}(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)\s(STREET|ST|DRIVE|DR|AVENUE|AVE|ROAD|RD|LOOP|COURT|CT|CIRCLE|LANE|LN|BOULEVARD|BLVD|COVE|TERRACE|TERR|TER|PLACE)\.?\,?(\s(APARTMENT|APT|BUILDING|BLDG|DEPARTMENT|DEPT|FLOOR|FL|LOT|PIER|ROOM|RM|TRAILER|TRLR|UNIT|PO BOX)\.?\s\#?[A-Za-z0-9]+\,?)?(\s[A-Za-z]+)?(\s[A-Za-z]+)\,?\s(A[LKSZR]|C[AOT]|D[EC]|F[L]|G[AU]|HI|I[ADLN]|K[SY]|LA|M[ADEINOPST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])(\s\d{5})?(\-\d{4})?', case=False)== True

reviews['regex_USaddress_headline'] = reviews['REVIEW_HEADLINE']. \
str.contains(r'\d{1,6}(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)\s(STREET|ST|DRIVE|DR|AVENUE|AVE|ROAD|RD|LOOP|COURT|CT|CIRCLE|LANE|LN|BOULEVARD|BLVD|COVE|TERRACE|TERR|TER|PLACE)\.?\,?(\s(APARTMENT|APT|BUILDING|BLDG|DEPARTMENT|DEPT|FLOOR|FL|LOT|PIER|ROOM|RM|TRAILER|TRLR|UNIT|PO BOX)\.?\s\#?[A-Za-z0-9]+\,?)?(\s[A-Za-z]+)?(\s[A-Za-z]+)\,?\s(A[LKSZR]|C[AOT]|D[EC]|F[L]|G[AU]|HI|I[ADLN]|K[SY]|LA|M[ADEINOPST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])(\s\d{5})?(\-\d{4})?', case=False)== True


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib64/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [20]:
reviews['regex_PII'] = 0
reviews.loc[(reviews['regex_cc']==True) | (reviews['regex_socsec']==True) | (reviews['regex_phone10']==True) \
         | (reviews['regex_email']==True) | (reviews['regex_handle']==True) | (reviews['regex_USaddress_loc']==True) \
        | (reviews['regex_USaddress_comments']==True) | (reviews['regex_USaddress_headline']==True), 'regex_PII'] = 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
FP = reviews[(reviews['PII'] == 0) & (reviews['regex_PII'] == 1)]
FN = reviews[(reviews['PII'] == 1) & (reviews['regex_PII'] == 0)]
TP = reviews[(reviews['PII'] == 1) & (reviews['regex_PII'] == 1)]
TN = reviews[(reviews['PII'] == 0) & (reviews['regex_PII'] == 0)]
FP['UGC_ID'].count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

18031

In [22]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

115531

In [23]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6910

In [24]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16409912

In [35]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

test = FP[['ALL_TEXT','PII','regex_PII']]
test.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEWER_NICKNAME", StringType(), True)\
,StructField("REVIEWER_LOCATION", StringType(), True)\
,StructField("PII", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("regex_cc", BooleanType(), True)\
,StructField("regex_soc_sec", BooleanType(), True)\
,StructField("regex_phone10", BooleanType(), True)\
,StructField("regex_email", BooleanType(), True)\
,StructField("regex_handle", BooleanType(), True)\
,StructField("regex_USaddress_loc", BooleanType(), True)\
,StructField("regex_USaddress_comments", BooleanType(), True)\
,StructField("regex_USaddress_headline", BooleanType(), True)\
,StructField("regex_PII", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","PII_FP_test").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Ok if it doesn't get phone # with 7 digits bc no one could call without area code anyway
#Would mark any location with Name, only want addresses with street info in them
text = "My name is Apple Smith can be contacted on example@example.com, or 1800 555-5555 or 123-3421 or +1(800)452-1234 \
soc sec num 123-02-3240 or without dashes 123023240 or with periods 123.02.3240 \
or at 414 Wrightwood Ave Chicago, IL or in Chicago or in Chicago, IL. \
I work at The North Face. Is this a Name? \
Credit card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. \
Website is http://www.amazon.com or www.amazon.com or amazon.com"
 

# Replaces the phone number and email addresse with anonymous IDs.
scrubadub.clean(text)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'My name is {{NAME}} {{NAME}} can be contacted on {{EMAIL}}, or {{PHONE}} or 123-3421 or {{PHONE}} soc sec num {{SSN}} or without dashes 123023240 or with periods {{SSN}} or at 414 {{NAME}} {{NAME}} {{NAME}}, IL or in {{NAME}} or in {{NAME}}, IL. I work at The {{NAME}} {{NAME}}. Is this a Name? {{NAME}} card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. {{NAME}} is {{URL}} or {{URL}} or amazon.com'

In [9]:
scrubber = scrubadub.Scrubber()
scrubber.remove_detector('name')
scrubber.clean(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'My name is Apple Smith can be contacted on {{EMAIL}}, or {{PHONE}} or 123-3421 or {{PHONE}} soc sec num {{SSN}} or without dashes 123023240 or with periods {{SSN}} or at 414 Wrightwood Ave Chicago, IL or in Chicago or in Chicago, IL. I work at The North Face. Is this a Name? Credit card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. Website is {{URL}} or {{URL}} or amazon.com'

In [14]:
txt = "My credit card is 1234 123412341234"
x = re.search('\d{4}(-|\.|\s)?\d{4}(-|\.|\s)?\d{4}(-|\.|\s)?\d{4}', txt)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<re.Match object; span=(18, 35), match='1234 123412341234'>

In [ ]:
conn.cursor().execute("drop table analytics_workshop.data_science.PII_FP_test1")